# Dataset Construction

This Notebook uses SEC's free EDGAR API to pull fundamental data and IBKR's free TWS API to pull price data. We then calculate the various Value, Quality and Momentum metrics and output it in a full_factor_table.csv.

## Section 1: Preparation

### 1.1 Importing Libraries and Stock Universe

In [1]:
import time
import requests
import numpy as np
import pandas as pd
import nest_asyncio
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from pathlib import Path
from ib_insync import IB, Stock, util
from datetime import datetime, timedelta
from scipy.stats import ttest_1samp
from typing import Optional
import locale

In [2]:
# ── 1. Settings ────────────────────────────────────────────────
EDGAR_API_KEY    = 'fc6b859f477a845e0b2a214bfc4cddf78c86f92d5d764845367b6484e7b60d41'
START_DATE     = '2010-01-01'
END_DATE       = '2025-05-31 23:59:59'
locale.setlocale(locale.LC_ALL, '')  # use your locale, e.g. 'en_US.UTF-8'

top100_symbols = [
    "MSFT","AAPL","NVDA","AMZN","AVGO","TSLA","GOOGL", "APP", "MO",
    "JPM","V","LLY","NFLX","XOM","MA","COST","WMT","PG","HD", "BMY",
    "JNJ","ABBV","BAC","UNH","CRM","KO","ORCL","PM","WFC", "INTC",
    "CSCO","IBM","CVX","GE","ABT","MCD","NOW","ACN","DIS", "COF", "PH",
    "MRK","UBER","T","GS","INTU","AMD","VZ","PEP","BKNG", "CEG", "CVS",
    "RTX","ADBE","TXN","CAT","AXP","QCOM","PGR","TMO","SPGI","MS",
    "BA","BSX","NEE","TJX","SCHW","AMGN","HON","C","AMAT", "NEM",
    "UNP","SYK","CMCSA","ETN","LOW","PFE","GILD","DE","DHR", "LMT",
    "ADP","COP","GEV","TMUS","ADI","MMC","LRCX","MDT", "HCA",
    "MU","CB","KLAC","APH","ANET","ICE","SBUX", "CMCSA", "MCK"
]

iwm_top100 = [
    "BE","CRDO","FN","IONQ","SATS","NXT","KTOS","GH","HL","BBIO",
    "RVMD","OKLO","CDE","SPXC","MDGL","U","DY","STRL","ENSG","GTLS",
    "IDCC","SANM","UMBF","MOD","QBTS","RGTI","AVAV","ALKS","HQY","DNA",
    "UMPS","STEM","CMC","IPAR","TTMI","AKRV","WNS","CTRS","AIR","PUB",
    "PLR","RXA","AXGN","HIMS","ZWS","JDGY","CVLT","ORA","SMTC","PS",
    "CADE","IBP","JXN","APLD","FCFS","PIPR","BOOT","BNT","TND","EAT",
    "CWST","MIR","TMHC","RYTM","KSPI","GDS","GATX","BRKT","ASGN","LIC",
    "LEO","VLY","PTCT","INIC","HWC","TOWN","GBCI","LXP","BIPC","LNG",
    "QLYS","CVLT","COST","PRAE","HOMA","CNC","RIOT","ACA","OPCH","BMI",
    "BXC","GPI","INTC","CWAN","PGNY","KNSL","ACHR","POR","BJH"
]

sp500_tickers = top100_symbols
len(sp500_tickers)

100

### 1.2 Fetching Fundamentals Data from SEC

For each ticker, we find its Central Index Key (CIK) and then use that to query the SEC database. We fetch quarterly fundamental data through the EDGAR API using the XRBL tags. This is done on a trailing-twelve-month basis so as to deal with the different financial statement filing types (10Q vs 10K). Moreover, as financial statement items can have different names in different companies, we query a wide dictionary of similar words.

We then construct monthly windows as of the first trading day (approx. first business day) of each month, using the nearest *prior* TTM fundamentals for each ticker.

In [3]:
def _get_cik_from_ticker(
    ticker: str,
    user_agent: str,
    api_key: Optional[str] = None
) -> str:
    """
    Resolve ticker -> CIK using SEC's company_tickers.json file.
    Returns a zero-padded 10-digit string.
    """
    base_url = "https://www.sec.gov/files/company_tickers.json"
    headers = {
        "User-Agent": user_agent,
        "Accept-Encoding": "gzip, deflate",
    }
    if api_key:
        headers["X-API-KEY"] = api_key

    resp = requests.get(base_url, headers=headers)
    resp.raise_for_status()
    data = resp.json()

    ticker = ticker.upper()
    cik_str = None
    for entry in data.values():
        if entry.get("ticker", "").upper() == ticker:
            cik_int = entry["cik_str"]
            cik_str = str(cik_int).zfill(10)
            break

    if cik_str is None:
        raise ValueError(f"Could not find CIK for ticker {ticker}")

    return cik_str


def fetch_edgar_fundamentals(
    ticker: str,
    user_agent: str,
    api_key: Optional[str] = None
) -> pd.DataFrame:
    """
    Fetch fundamentals from SEC EDGAR companyfacts API and return
    a quarterly-only dataframe with TTM metrics (using EBIT instead of EBITDA).
    `period_type` is removed from the final output.
    """
    # --- 1) Resolve ticker -> CIK ---
    cik_str = _get_cik_from_ticker(ticker, user_agent=user_agent, api_key=api_key)

    # --- 2) Fetch companyfacts JSON ---
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik_str}.json"
    headers = {
        "User-Agent": user_agent,
        "Accept-Encoding": "gzip, deflate",
    }
    if api_key:
        headers["X-API-KEY"] = api_key

    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    data = resp.json()
    facts = data.get("facts", {}).get("us-gaap", {})

    # --- 3) Tag mapping (EXPANDED DEBT + MARGINS) ---
    tag_map = {
        "total_equity": [
            "StockholdersEquity",
            "StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest",
            "Equity",
            "PartnersCapital",
            "MembersEquity",
        ],

        # >>> EXPANDED LONG-TERM DEBT (NONCURRENT) <<<
        "long_term_debt_noncurrent": [
            "LongTermDebtNoncurrent",
            "DebtNoncurrent",
            "LongTermBorrowings",
            "LongTermDebtAndCapitalLeaseObligations",
            "LongTermDebtAndFinanceLeaseObligations",
            "LongTermDebt",
            "LongTermLoans",
            "NotesPayableNoncurrent",
            "LoansPayableNoncurrent",
            "ConvertibleDebtNoncurrent",
            "UnsecuredDebtNoncurrent",
            "SecuredDebtNoncurrent",
            "SeniorLongTermDebt",
            "SubordinatedLongTermDebt",
            "MortgageNotesPayableNoncurrent",
            "DebenturesNoncurrent",
            "CommercialPaperNoncurrent",
            "FinanceLeaseLiabilityNoncurrent",
            "CapitalLeaseObligationsNoncurrent",
            "DebtInstrumentCarryingAmount",
            "LongTermPortionOfDebt",
            "LongTermPortionOfBorrowings",
        ],

        # >>> EXPANDED CURRENT DEBT <<<
        "long_term_debt_current": [
            "LongTermDebtCurrent",
            "DebtCurrent",
            "CurrentPortionOfLongTermDebt",
            "CurrentPortionOfLongTermBorrowings",
            "CurrentPortionOfNotesPayable",
            "CurrentPortionOfDebt",
            "CurrentPortionOfBorrowings",
            "CurrentPortionOfBankLoans",
            "CurrentPortionOfConvertibleDebt",
            "CurrentPortionOfFinanceLeaseLiability",
            "CurrentPortionOfCapitalLeaseObligation",
            "ShortTermBorrowings",
            "ShortTermNotesPayable",
            "BankOverdrafts",
            "CommercialPaper",
            "DebtCurrentExcludingFinanceLeases",
        ],

        "cash_and_equivalents": [
            "CashAndCashEquivalentsAtCarryingValue",
            "CashCashEquivalentsAndShortTermInvestments",
            "CashAndCashEquivalents",
        ],

        "revenue": [
            "RevenueFromContractWithCustomerExcludingAssessedTax",
            "Revenues",
            "Revenue",
            "SalesRevenueNet",
            "SalesRevenueGoodsNet",
            "ProductRevenue",
            "ServiceRevenue",
        ],

        # >>> EXPANDED GROSS PROFIT TAGS <<<
        "gross_profit": [
            "GrossProfit",
            "GrossProfitLoss",
            "OperatingRevenueLessCostOfRevenue",
            "RevenuesLessCostOfSales",
        ],

        # >>> EXPANDED OPERATING INCOME TAGS (EBIT PROXY) <<<
        "operating_income": [
            "OperatingIncomeLoss",
            "IncomeLossFromOperations",
            "OperatingProfitLoss",
            "OperatingProfit",
            "IncomeFromOperations",
            "OperatingIncomeLossContinuingOperations",
            "OperatingIncomeLossContinuingOperationsAndIncomeLossFromEquityMethodInvestments",
            # fallback: pre-tax income from continuing ops if no clean operating income tag
            "IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest",
        ],

        "net_income": [
            "NetIncomeLoss",
            "ProfitLoss",
            "NetIncomeLossAvailableToCommonStockholdersBasic",
        ],

        # we keep these tags in case you want them later, but we won't build EBITDA anymore
        "dep_amort": [
            "DepreciationDepletionAndAmortization",
            "DepreciationAndAmortization",
            "DepreciationAndAmortizationOfPropertyPlantAndEquipment",
        ],

        "interest_expense": [
            "InterestExpense",
            "InterestAndDebtExpense",
            "InterestExpenseBorrowings",
        ],

        "income_tax_expense": [
            "IncomeTaxExpenseBenefit",
            "IncomeTaxExpense",
            "IncomeTaxProvision",
        ],

        "shares_outstanding": [
            "CommonStockSharesOutstanding",
            "EntityCommonStockSharesOutstanding",
            "WeightedAverageNumberOfSharesOutstandingBasic",
        ],
    }

    # --- Raw period collector ---
    periods = {}

    def _add_fact_series(field_name, tag_candidates, unit_preference=("USD", "shares")):
        for tag in tag_candidates:
            if tag not in facts:
                continue
            units_dict = facts[tag].get("units", {})
            for unit in unit_preference:
                if unit not in units_dict:
                    continue

                for item in units_dict[unit]:
                    end = item.get("end")
                    val = item.get("val")
                    form = item.get("form", "")
                    fp   = item.get("fp", "")
                    fy   = item.get("fy", None)

                    if end is None or val is None:
                        continue

                    try:
                        dt = datetime.fromisoformat(end)
                    except Exception:
                        continue

                    key = dt.date()
                    if key not in periods:
                        periods[key] = {
                            "fiscal_period_end": dt.date(),
                            "form": form,
                            "fp": fp,
                            "fy": fy,
                        }

                    if field_name not in periods[key] or periods[key][field_name] is None:
                        periods[key][field_name] = val

                break

    # Fill all fields
    for field_name, tag_candidates in tag_map.items():
        if field_name == "shares_outstanding":
            _add_fact_series(field_name, tag_candidates, unit_preference=("shares", "SHARES"))
        else:
            _add_fact_series(field_name, tag_candidates, unit_preference=("USD",))

    if not periods:
        raise ValueError("No EDGAR facts found for this ticker.")

    df = pd.DataFrame(list(periods.values()))
    df["fiscal_period_end"] = pd.to_datetime(df["fiscal_period_end"])

    # --- Total Debt ---
    lt_non = df.get("long_term_debt_noncurrent", np.nan)
    lt_cur = df.get("long_term_debt_current", np.nan)
    df["total_debt"] = lt_non.fillna(0) + lt_cur.fillna(0)
    df.loc[df["total_debt"] == 0, "total_debt"] = np.nan

    # --- Determine Period Type ---
    def _ptype(r):
        fp = str(r.get("fp", "")).upper()
        form = str(r.get("form", "")).upper()
        if fp == "FY" or form in ("10-K", "20-F", "40-F"):
            return "annual"
        return "quarterly"

    df["period_type"] = df.apply(_ptype, axis=1)
    df["fy"] = df["fy"].fillna(df["fiscal_period_end"].dt.year)

    # --- EBIT construction (from operating_income) ---
    # Treat operating_income as EBIT proxy; if missing, ebit = NaN
    if "operating_income" not in df.columns:
        df["operating_income"] = np.nan
    df["ebit"] = df["operating_income"].astype(float)

    # --- Q1–Q4 Construction ---
    # Now using EBIT instead of EBITDA in the flow set
    flow_cols  = ["revenue", "gross_profit", "operating_income", "ebit", "net_income"]
    stock_cols = ["total_equity", "total_debt", "cash_and_equivalents", "shares_outstanding"]

    df_q = df[df["period_type"] == "quarterly"].copy()
    df_fy = df[df["period_type"] == "annual"].copy()

    q4_rows = []

    for fy, grp in df_fy.groupby("fy"):
        fy_row = grp.sort_values("fiscal_period_end").iloc[-1]
        qrows = df_q[df_q["fy"] == fy].sort_values("fiscal_period_end")

        qmap = {}
        for _, r in qrows.iterrows():
            fp = str(r.get("fp", "")).upper()
            if fp in ("Q1", "Q2", "Q3") and fp not in qmap:
                qmap[fp] = r

        if len(qmap) < 3 and len(qrows) >= 3:
            rt = list(qrows.itertuples())
            qmap = {"Q1": rt[0], "Q2": rt[1], "Q3": rt[2]}

        if len(qmap) < 3:
            continue

        q1, q2, q3 = qmap["Q1"], qmap["Q2"], qmap["Q3"]

        q4 = {
            "fiscal_period_end": fy_row["fiscal_period_end"],
            "period_type": "quarterly",
            "fp": "Q4",
            "fy": fy,
        }

        for col in stock_cols:
            q4[col] = fy_row.get(col, np.nan)

        for col in flow_cols:
            fyv = fy_row.get(col, np.nan)
            if pd.isna(fyv):
                q4[col] = np.nan
            else:
                parts = [getattr(q1, col, np.nan), getattr(q2, col, np.nan), getattr(q3, col, np.nan)]
                if any(pd.isna(p) for p in parts):
                    q4[col] = np.nan
                else:
                    q4[col] = float(fyv) - sum(float(p) for p in parts)

        q4_rows.append(q4)

    if q4_rows:
        df_q4 = pd.DataFrame(q4_rows)
        df_q = pd.concat([df_q, df_q4], ignore_index=True)

    # --- Compute TTM (including ebit_ttm) ---
    df_q = df_q.sort_values("fiscal_period_end").reset_index(drop=True)
    for col in flow_cols:
        df_q[col] = df_q[col].astype(float)
        df_q[col + "_ttm"] = df_q[col].rolling(window=4, min_periods=4).sum()

    # --- Final Output (drop period_type) ---
    base_cols = [
        "fiscal_period_end",
        "total_equity", "total_debt", "cash_and_equivalents",
        "revenue", "gross_profit", "operating_income", "ebit",
        "net_income", "shares_outstanding",
    ]
    ttm_cols = [c + "_ttm" for c in flow_cols]  # includes ebit_ttm

    for col in base_cols:
        if col not in df_q.columns:
            df_q[col] = np.nan

    fund_df = df_q[base_cols + ttm_cols].sort_values("fiscal_period_end").reset_index(drop=True)
    return fund_df

In [4]:
# --- CONFIG ---
USER_AGENT = "kampongpiggg@gmail.com"  # <-- replace with your SEC-compliant UA

# --- LOOP THROUGH TICKERS AND CONCATENATE FUNDAMENTALS ---
all_fund_dfs = []

for ticker in sp500_tickers:
    try:
        print(f"Fetching fundamentals for {ticker} ...")
        fund_df = fetch_edgar_fundamentals(
            ticker=ticker,
            user_agent=USER_AGENT,
            api_key=EDGAR_API_KEY,
        )
        fund_df = fund_df.copy()
        fund_df["ticker"] = ticker  # tag rows with ticker
        all_fund_dfs.append(fund_df)
    except Exception as e:
        print(f"⚠️ Failed to fetch {ticker}: {e}")

if all_fund_dfs:
    all_fund_df = pd.concat(all_fund_dfs, ignore_index=True)
    print("✅ Combined fundamentals shape:", all_fund_df.shape)
else:
    all_fund_df = pd.DataFrame()
    print("⚠️ No data frames were created; check errors above.")

# (Optional) Save to CSV
all_fund_df.to_csv("all_fundamentals_ttm.csv", index=False)
print("Saved combined fundamentals to all_fundamentals_ttm.csv")

Fetching fundamentals for MSFT ...
Fetching fundamentals for AAPL ...
Fetching fundamentals for NVDA ...
Fetching fundamentals for AMZN ...
Fetching fundamentals for AVGO ...
Fetching fundamentals for TSLA ...
Fetching fundamentals for GOOGL ...
Fetching fundamentals for APP ...
Fetching fundamentals for MO ...
Fetching fundamentals for JPM ...
Fetching fundamentals for V ...
Fetching fundamentals for LLY ...
Fetching fundamentals for NFLX ...
Fetching fundamentals for XOM ...
Fetching fundamentals for MA ...
Fetching fundamentals for COST ...
Fetching fundamentals for WMT ...
Fetching fundamentals for PG ...
Fetching fundamentals for HD ...
Fetching fundamentals for BMY ...
Fetching fundamentals for JNJ ...
Fetching fundamentals for ABBV ...
Fetching fundamentals for BAC ...
Fetching fundamentals for UNH ...
Fetching fundamentals for CRM ...
Fetching fundamentals for KO ...
Fetching fundamentals for ORCL ...
Fetching fundamentals for PM ...
Fetching fundamentals for WFC ...
Fetching f

In [5]:
def prepare_monthly_fundamentals(
    csv_path: str = "all_fundamentals_ttm.csv",
    freq: str = "BMS"  # Business Month Start = first business day of each month
) -> pd.DataFrame:
    """
    Load fundamentals and create a monthly 'as-of' panel on the first trading
    day (approx. first business day) of each month, using the nearest
    *prior* TTM fundamentals for each ticker.

    Steps:
    1) Read CSV `all_fundamentals_ttm`.
    2) Convert fiscal_period_end to datetime.
    3) For each ticker:
       - Build a date range from min to max fiscal_period_end with freq='BMS'.
       - For each monthly date, attach the latest fundamentals row whose
         fiscal_period_end <= as-of date (merge_asof with direction='backward').

    Returns
    -------
    monthly_fund_df : DataFrame
        Indexed by ['ticker', 'asof_date'].
        All original fundamental columns are preserved.
    """
    # 1) Read CSV
    df = pd.read_csv(csv_path)

    # 2) Convert fiscal_period_end to datetime and sort
    df["fiscal_period_end"] = pd.to_datetime(df["fiscal_period_end"])
    df = df.sort_values(["ticker", "fiscal_period_end"])

    out_frames = []

    # 3) Process each ticker independently
    for ticker, grp in df.groupby("ticker"):
        grp = grp.sort_values("fiscal_period_end")

        # Build monthly index: first business day of each month
        start = grp["fiscal_period_end"].min().normalize()
        end = grp["fiscal_period_end"].max().normalize()
        monthly_idx = pd.date_range(start=start, end=end, freq=freq)

        monthly_dates = pd.DataFrame({"asof_date": monthly_idx})

        # For each as-of date, get the last available fundamentals row
        # with fiscal_period_end <= asof_date
        merged = pd.merge_asof(
            monthly_dates.sort_values("asof_date"),
            grp.sort_values("fiscal_period_end"),
            left_on="asof_date",
            right_on="fiscal_period_end",
            direction="backward"
        )

        merged["ticker"] = ticker
        out_frames.append(merged)

    monthly_fund_df = pd.concat(out_frames, ignore_index=True)

    # Set a clean MultiIndex: (ticker, asof_date)
    monthly_fund_df = monthly_fund_df.set_index(["ticker", "asof_date"]).sort_index()

    return monthly_fund_df

monthly_fund_df = prepare_monthly_fundamentals()

### 1.3 Fetching OHLCV Price Data from IBKR

We use IBKR's historical data and query price data. We fetch OHLCV price data, which includes open, high, low, close and volume data.

In [6]:
# ── 4. Fetch Price & Volume from IBKR ─────────────────────────
def fetch_ohlcv_ibkr(tickers):

    nest_asyncio.apply()
    ib = IB()
    ib.connect('127.0.0.1', 7496, clientId=21)
    dfs = []
    
    for symbol in tickers:
        contract = Stock(symbol, 'SMART', 'USD')
        bars = ib.reqHistoricalData(
            contract,
            endDateTime = '',
            durationStr='15 Y',
            barSizeSetting='1 day',
            whatToShow='TRADES',
            useRTH=True
        )
        df = util.df(bars)[['date','open','high','low','close','volume']]
        df['ticker'] = symbol
        dfs.append(df)
    
    ib.disconnect()
    return pd.concat(dfs, ignore_index=True)

start = time.time()
raw_df = fetch_ohlcv_ibkr(sp500_tickers)
end   = time.time()
print(f"Price data pulled: {raw_df.shape[0]} rows in {((end-start)/60):.1f} mins")

Price data pulled: 359217 rows in 11.7 mins


## Section 2: Feature Computation

We then computed the different Value, Quality and Momentum metrics.

### 2.1 Computing Value Factors

We are primarily interested in three Value metrics:
* book_to_price &rarr; shows how overvalued / undervalued the companies' assets less liabilities are compared to the market price of the shares.
* ev_ebit &rarr; shows how much investors pay per dollar of core earnings
* debt_to_equity &rarr; shows how much financial leverage the company's using (correlates to health)

We calculate the metrics above manually using the fundamental data extracted.

In [7]:
def compute_value_factors(
    price_df: pd.DataFrame,
    fund_df: pd.DataFrame,
    date_range,
    report_lag_days: int = 90
):
    """
    Compute value factors over `date_range`:
      - book_to_price
      - ev_ebit (using TTM EBIT)
      - debt_to_equity

    Parameters
    ----------
    price_df : daily price/volume DataFrame
               index: DatetimeIndex of trading days
               columns: ['close','volume',...]
    fund_df  : fundamentals DataFrame
               must contain at least:
               ['fiscal_period_end','total_equity','total_debt',
                'cash_and_equivalents','ebit_ttm','shares_outstanding']
    date_range : iterable of datetime-like (rebalance dates)
    report_lag_days : int, lag in days to mimic reporting delay

    Returns
    -------
    value_df : DataFrame indexed by date_range with columns:
               ['book_to_price','ev_ebit','debt_to_equity']
    """
    date_index = pd.to_datetime(pd.Index(date_range)).sort_values()
    value_rows = []

    for asof in date_index:
        asof = pd.to_datetime(asof)

        # --- price as of date (last available) ---
        px_idx = price_df.index[price_df.index <= asof]
        if len(px_idx) == 0:
            value_rows.append({
                "date": asof,
                "book_to_price": np.nan,
                "ev_ebit": np.nan,
                "debt_to_equity": np.nan,
            })
            continue

        # force scalar
        price_val = price_df.loc[px_idx[-1], "close"]
        try:
            price_val = float(price_val)
        except (TypeError, ValueError):
            price_val = np.nan

        # --- fundamentals with reporting lag ---
        cutoff = asof - timedelta(days=report_lag_days)
        eligible = fund_df[fund_df["fiscal_period_end"] <= cutoff]

        if eligible.empty:
            value_rows.append({
                "date": asof,
                "book_to_price": np.nan,
                "ev_ebit": np.nan,
                "debt_to_equity": np.nan,
            })
            continue

        f = eligible.iloc[-1]

        # pull & coerce to scalars
        def _to_float(x, default=np.nan):
            if x is None:
                return default
            if pd.isna(x):
                return default
            try:
                return float(x)
            except (TypeError, ValueError):
                return default

        equity_val = _to_float(f.get("total_equity", np.nan))
        debt_val   = _to_float(f.get("total_debt", np.nan))
        cash_val   = _to_float(f.get("cash_and_equivalents", 0.0))
        ebit_ttm   = _to_float(f.get("ebit_ttm", np.nan))
        shares_val = _to_float(f.get("shares_outstanding", np.nan))

        # --- book_to_price ---
        if pd.notna(equity_val) and pd.notna(shares_val) and pd.notna(price_val) and shares_val != 0 and price_val != 0:
            book_per_share = equity_val / shares_val
            book_to_price = book_per_share / price_val
        else:
            book_to_price = np.nan

        # --- EV / EBIT (TTM) ---
        if pd.notna(price_val) and pd.notna(shares_val):
            mkt_cap = price_val * shares_val
        else:
            mkt_cap = np.nan

        if pd.notna(mkt_cap) and pd.notna(ebit_ttm) and ebit_ttm != 0:
            # treat missing debt/cash as 0 in EV bridge
            ev = mkt_cap + (debt_val if pd.notna(debt_val) else 0.0) - (cash_val if pd.notna(cash_val) else 0.0)
            ev_ebit = ev / ebit_ttm
        else:
            ev_ebit = np.nan

        # --- debt_to_equity ---
        if pd.notna(debt_val) and pd.notna(equity_val) and equity_val != 0:
            debt_to_equity = debt_val / equity_val
        else:
            debt_to_equity = np.nan

        value_rows.append({
            "date": asof,
            "book_to_price": book_to_price,
            "ev_ebit": ev_ebit,
            "debt_to_equity": debt_to_equity,
        })

    value_df = pd.DataFrame(value_rows).set_index("date").sort_index()
    return value_df


### 2.2 Compute Quality Factors

We are primarily interested in three Quality metrics:
* gross_margin &rarr; shows the core profitability of the business's products (does not include overheads).
* operating_margin &rarr; shows the core profitability of a business's operations (includes overheads).
* roe &rarr; shows management's efficiency at generating returns.

We calculate the metrics above manually using the fundamental data extracted.

In [8]:
def compute_quality_factors(
    fund_df: pd.DataFrame,
    date_range,
    report_lag_days: int = 90
):
    """
    Compute quality factors over `date_range`:
      - gross_margin  = gross_profit / revenue
      - operating_margin = operating_income / revenue
      - roe = net_income / total_equity

    Parameters
    ----------
    fund_df  : fundamentals DataFrame
    date_range : iterable of datetime-like
    report_lag_days : int

    Returns
    -------
    quality_df : DataFrame indexed by date_range with columns:
                 ['gross_margin','operating_margin','roe']
    """
    date_index = pd.to_datetime(pd.Index(date_range)).sort_values()
    quality_rows = []

    # small helper to coerce to float scalars
    def _to_float(x, default=np.nan):
        if x is None:
            return default
        if pd.isna(x):
            return default
        try:
            return float(x)
        except (TypeError, ValueError):
            return default

    for asof in date_index:
        asof = pd.to_datetime(asof)
        cutoff = asof - timedelta(days=report_lag_days)
        eligible = fund_df[fund_df["fiscal_period_end"] <= cutoff]

        if eligible.empty:
            quality_rows.append({
                "date": asof,
                "gross_margin": np.nan,
                "operating_margin": np.nan,
                "roe": np.nan,
            })
            continue

        f = eligible.iloc[-1]

        # coerce all inputs to clean scalars
        revenue       = _to_float(f.get("revenue", np.nan))
        gross_profit  = _to_float(f.get("gross_profit", np.nan))
        op_inc        = _to_float(f.get("operating_income", np.nan))
        net_inc       = _to_float(f.get("net_income", np.nan))
        equity        = _to_float(f.get("total_equity", np.nan))

        # --- gross_margin ---
        if pd.notna(gross_profit) and pd.notna(revenue) and revenue != 0:
            gross_margin = gross_profit / revenue
        else:
            gross_margin = np.nan

        # --- operating_margin ---
        if pd.notna(op_inc) and pd.notna(revenue) and revenue != 0:
            operating_margin = op_inc / revenue
        else:
            operating_margin = np.nan

        # --- ROE ---
        if pd.notna(net_inc) and pd.notna(equity) and equity != 0:
            roe = net_inc / equity
        else:
            roe = np.nan

        quality_rows.append({
            "date": asof,
            "gross_margin": gross_margin,
            "operating_margin": operating_margin,
            "roe": roe,
        })

    quality_df = pd.DataFrame(quality_rows).set_index("date").sort_index()
    return quality_df

### 2.3 Compute Momentum Factors

We are primarily interested in two Momentum metrics:
* mom_6m &rarr; shows the raw price trend strength in the last six months (how much has it moved in the last six months?)
* obv &rarr; shows volume-confirmed momentum; rising OBV indicates accumulation and buying pressure, while falling OBV indicates distribution and selling pressure.

We calculate the metrics above manually using the price data extracted.

In [9]:
def compute_momentum_factors(
    price_df: pd.DataFrame,
    date_range,
    mom_window_days: int = 126,   # ~6 months of trading days
    obv_lookback_days: int = 252  # 1Y for OBV build; can tune
):
    """
    Compute momentum factors over `date_range`:
      - mom_6m : 6-month price momentum
      - obv    : On-Balance Volume (absolute level at as-of date)

    Parameters
    ----------
    price_df : daily price DataFrame (must have 'close','volume')
    date_range : iterable of datetime-like
    mom_window_days : lookback window for momentum (in calendar days)
    obv_lookback_days : lookback window for OBV construction (in calendar days)

    Returns
    -------
    mom_df : DataFrame indexed by date_range with columns:
             ['mom_6m','obv']
    """
    price_df = price_df.sort_index()
    date_index = pd.to_datetime(pd.Index(date_range)).sort_values()
    mom_rows = []

    # Precompute OBV over full history once
    obv_series = []
    obv = 0
    prev_close = None
    for dt, row in price_df.iterrows():
        c = row['close']
        v = row['volume']
        if prev_close is None:
            prev_close = c
            obv_series.append((dt, obv))
            continue
        if c > prev_close:
            obv += v
        elif c < prev_close:
            obv -= v
        # else, OBV unchanged
        prev_close = c
        obv_series.append((dt, obv))

    obv_df = pd.DataFrame(obv_series, columns=['dt', 'obv']).set_index('dt')

    for asof in date_index:
        asof = pd.to_datetime(asof)

        # Align to last trading day <= asof
        px_idx = price_df.index[price_df.index <= asof]
        if len(px_idx) == 0:
            mom_rows.append({'date': asof, 'mom_6m': np.nan, 'obv': np.nan})
            continue
        asof_trading = px_idx[-1]

        # --- 6m momentum ---
        mom_start = asof_trading - timedelta(days=mom_window_days)
        hist_mom = price_df.loc[mom_start:asof_trading]

        if len(hist_mom) < 2:
            mom_6m = np.nan
        else:
            p_t = hist_mom['close'].iloc[-1]
            p_0 = hist_mom['close'].iloc[0]
            mom_6m = p_t / p_0 - 1

        # --- OBV at as-of date ---
        obv_idx = obv_df.index[obv_df.index <= asof_trading]
        if len(obv_idx) == 0:
            obv_val = np.nan
        else:
            obv_val = obv_df.loc[obv_idx[-1], 'obv']

        mom_rows.append({
            'date': asof,
            'mom_6m': mom_6m,
            'obv': obv_val
        })

    mom_df = pd.DataFrame(mom_rows).set_index('date').sort_index()
    return mom_df

### 2.4 Building Factor Table

We then build the entire factor table by running the three helper functions above in a loop. The result is a concatenated table with all the metrics indexed by (ticker, date).

In [10]:
def build_factor_table(
    raw_df: pd.DataFrame,
    monthly_fund_df: pd.DataFrame,
    tickers=None,
    report_lag_days: int = 90,
    mom_window_days: int = 126,
):
    """
    Build factor tables for all tickers using:
      - raw_df: OHLCV for multiple tickers
                columns must include ['date','close','volume','ticker']
      - monthly_fund_df: fundamentals with MultiIndex ['ticker','asof_date']
                         and at least 'fiscal_period_end' plus the
                         other fields required by the helpers.

    Returns
    -------
    factors_all : DataFrame
        MultiIndex (ticker, date) with columns:
        ['book_to_price','ev_ebitda','debt_to_equity',
         'gross_margin','operating_margin','roe',
         'mom_6m','obv']
    """

    # ----- Ensure date column is datetime -----
    raw_df = raw_df.copy()
    raw_df["date"] = pd.to_datetime(raw_df["date"])

    # ----- Infer tickers if not provided -----
    if tickers is None:
        tickers_price = set(raw_df["ticker"].unique())

        if isinstance(monthly_fund_df.index, pd.MultiIndex) and "ticker" in monthly_fund_df.index.names:
            tickers_fund = set(monthly_fund_df.index.get_level_values("ticker").unique())
        else:
            tickers_fund = set(monthly_fund_df["ticker"].unique())

        tickers = sorted(tickers_price & tickers_fund)

    all_factors = []

    for ticker in tickers:
        # ---------- slice price_df for this ticker ----------
        px = raw_df[raw_df["ticker"] == ticker].copy()
        if px.empty:
            continue

        price_df = (
            px.sort_values("date")
              .set_index("date")[["close", "volume"]]
        )

        # date_range comes directly from the price index (daily trading dates)
        date_range = pd.to_datetime(price_df.index.unique()).sort_values()

        # ---------- slice fund_df for this ticker ----------
        if isinstance(monthly_fund_df.index, pd.MultiIndex) and "ticker" in monthly_fund_df.index.names:
            fd = monthly_fund_df.xs(ticker, level="ticker").copy()
        else:
            fd = monthly_fund_df[monthly_fund_df["ticker"] == ticker].copy()

        if fd.empty:
            continue

        if "fiscal_period_end" not in fd.columns:
            raise ValueError("monthly_fund_df must contain a 'fiscal_period_end' column.")

        fund_df = fd.reset_index().sort_values("fiscal_period_end")

        # ---------- compute factor blocks ----------
        value_df = compute_value_factors(
            price_df=price_df,
            fund_df=fund_df,
            date_range=date_range,
            report_lag_days=report_lag_days,
        )

        quality_df = compute_quality_factors(
            fund_df=fund_df,
            date_range=date_range,
            report_lag_days=report_lag_days,
        )

        mom_df = compute_momentum_factors(
            price_df=price_df,
            date_range=date_range,
            mom_window_days=mom_window_days,
        )

        # ---------- combine for this ticker ----------
        df = pd.concat([value_df, quality_df, mom_df], axis=1)
        df["ticker"] = ticker

        df = (
            df.reset_index()          # 'date' (from helpers) back to column
              .set_index(["ticker", "date"])
              .sort_index()
        )

        all_factors.append(df)

    if not all_factors:
        return pd.DataFrame(
            columns=[
                "book_to_price", "ev_ebitda", "debt_to_equity",
                "gross_margin", "operating_margin", "roe",
                "mom_6m", "obv",
            ]
        )

    factors_all = pd.concat(all_factors, axis=0).sort_index()
    return factors_all

In [11]:
factors_table = build_factor_table(
    raw_df=raw_df,
    monthly_fund_df=monthly_fund_df,
)

In [12]:
# (Optional) Save to CSV
factors_table.to_csv("full_factors_table.csv", index=True)
print("Saved factor table to csv")

Saved factor table to csv
